In [0]:
! pip list | grep pyspark

In [2]:
! pip install pyspark

     |████████████████████████████████| 215.7MB 68kB/s 
     |████████████████████████████████| 204kB 49.1MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130387 sha256=8c9322f79f9f7c8386d8a255ec08a0663ee49312eb06b5ad54bd286265e1b10b
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark


In [0]:
from pyspark import *

In [0]:
sc = SparkContext(master="local", appName="test")

In [5]:
sc

<SparkContext master=local appName=test>

In [0]:
import urllib.request
response = urllib.request.urlopen('http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html')

In [0]:
import urllib
data = urllib.request.urlretrieve('http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz', 'data_10_percent.gz')
data_file = './data_10_percent.gz'

lines = sc.textFile(data_file)

In [8]:
lines

./data_10_percent.gz MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [9]:
lines.take(3)

['0,tcp,http,SF,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,9,9,1.00,0.00,0.11,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,19,19,1.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,235,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,29,29,1.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,normal.']

In [0]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [7]:
print(sqlContext)

In [12]:
lines.collect()

IllegalArgumentException: ignored

In [0]:
fonction_split = lambda x : x.split(',')

In [0]:
data_parsees = lines.map(fonction_split)

In [10]:
data_parsees.take(3)

[['0',
  'tcp',
  'http',
  'SF',
  '181',
  '5450',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '8',
  '8',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '1.00',
  '0.00',
  '0.00',
  '9',
  '9',
  '1.00',
  '0.00',
  '0.11',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  'normal.'],
 ['0',
  'tcp',
  'http',
  'SF',
  '239',
  '486',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '8',
  '8',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '1.00',
  '0.00',
  '0.00',
  '19',
  '19',
  '1.00',
  '0.00',
  '0.05',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  'normal.'],
 ['0',
  'tcp',
  'http',
  'SF',
  '235',
  '1337',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '8',
  '8',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '1.00',
  '0.00',
  '0.00',
  '29',
  '29',
  '1.00',
  '0.00',


In [0]:
from pyspark.sql.types import StructType

In [0]:
#from pyspark.sql.types import StructType
from pyspark.sql.types import *
schema = StructType([
    StructField("duration", IntegerType(), True),
    StructField("protocol_type", StringType(), True),
    StructField("service", StringType(), True),
    StructField("flag", StringType(), True),
    StructField("src_bytes",IntegerType(), True),
    StructField("dst_bytes",IntegerType(), True),
    StructField("interactions",StringType(), True),
    ])

In [13]:
schema

StructType(List(StructField(duration,IntegerType,true),StructField(protocol_type,StringType,true),StructField(service,StringType,true),StructField(flag,StringType,true),StructField(src_bytes,IntegerType,true),StructField(dst_bytes,IntegerType,true),StructField(interactions,StringType,true)))

In [14]:
newData = data_parsees.map(lambda key:(int(key[0]),key[1],key[2],key[3],int(key[4]),int(key[5]),key[-1]))
newData.take(3)

[(0, 'tcp', 'http', 'SF', 181, 5450, 'normal.'),
 (0, 'tcp', 'http', 'SF', 239, 486, 'normal.'),
 (0, 'tcp', 'http', 'SF', 235, 1337, 'normal.')]

In [15]:
dataFrame = sqlContext.createDataFrame(newData, schema)
dataFrame

DataFrame[duration: int, protocol_type: string, service: string, flag: string, src_bytes: int, dst_bytes: int, interactions: string]

In [16]:
dataFrame.show(10)


+--------+-------------+-------+----+---------+---------+------------+
|duration|protocol_type|service|flag|src_bytes|dst_bytes|interactions|
+--------+-------------+-------+----+---------+---------+------------+
|       0|          tcp|   http|  SF|      181|     5450|     normal.|
|       0|          tcp|   http|  SF|      239|      486|     normal.|
|       0|          tcp|   http|  SF|      235|     1337|     normal.|
|       0|          tcp|   http|  SF|      219|     1337|     normal.|
|       0|          tcp|   http|  SF|      217|     2032|     normal.|
|       0|          tcp|   http|  SF|      217|     2032|     normal.|
|       0|          tcp|   http|  SF|      212|     1940|     normal.|
|       0|          tcp|   http|  SF|      159|     4087|     normal.|
|       0|          tcp|   http|  SF|      210|      151|     normal.|
|       0|          tcp|   http|  SF|      212|      786|     normal.|
+--------+-------------+-------+----+---------+---------+------------+
only s

In [17]:
dataFrame.createTempView('interactions')
sqlContext.sql("SELECT interactions FROM interactions").show()

+------------+
|interactions|
+------------+
|     normal.|
|     normal.|
|     normal.|
|     normal.|
|     normal.|
|     normal.|
|     normal.|
|     normal.|
|     normal.|
|     normal.|
|     normal.|
|     normal.|
|     normal.|
|     normal.|
|     normal.|
|     normal.|
|     normal.|
|     normal.|
|     normal.|
|     normal.|
+------------+
only showing top 20 rows



In [0]:
# Une tempview est une photo (tuples) du résultat d'une requete sql executée sur la BDD
requete_bdd = sqlContext.sql("Select duration,dst_bytes from interactions where duration >1000 and dst_bytes = 0 and protocol_type='tcp'")

In [25]:
requete_bdd.count()





IllegalArgumentException: ignored

In [26]:
dataReq = requete_bdd.rdd.map(lambda rowItem: "Duration: {}, Dest. bytes: {}".format(rowItem.duration, rowItem.dst_bytes))
dataReq.take(dataReq.count())

IllegalArgumentException: ignored

In [0]:
from time import perf_counter

In [28]:
t1_start = perf_counter()  
sqlContext.sql("SELECT protocol_type, count('protocol_type') as count FROM interactions GROUP BY protocol_type").show()
t1_stop = perf_counter()  

+-------------+------+
|protocol_type| count|
+-------------+------+
|          tcp|190065|
|          udp| 20354|
|         icmp|283602|
+-------------+------+



In [53]:
dataframe2 = dataFrame.filter("duration>1000 AND dst_bytes = 0 AND protocol_type='tcp'" ).groupby("protocol_type").count().show()


+-------------+-----+
|protocol_type|count|
+-------------+-----+
|          tcp|  139|
+-------------+-----+



In [56]:
dataFrame.select(dataFrame.duration, dataFrame.dst_bytes).filter("duration>1000 AND dst_bytes = 0 AND protocol_type='tcp'").show()


+--------+---------+
|duration|dst_bytes|
+--------+---------+
|    5057|        0|
|    5059|        0|
|    5051|        0|
|    5056|        0|
|    5051|        0|
|    5039|        0|
|    5062|        0|
|    5041|        0|
|    5056|        0|
|    5064|        0|
|    5043|        0|
|    5061|        0|
|    5049|        0|
|    5061|        0|
|    5048|        0|
|    5047|        0|
|    5044|        0|
|    5063|        0|
|    5068|        0|
|    5062|        0|
+--------+---------+
only showing top 20 rows



In [0]:
from pyspark.sql.functions import udf
import pyspark.sql.functions as F

def attack_or_normal_func(s):
    return "normal" if s == "normal." else "attack"

In [64]:
udf_attOrNorm = F.udf(attack_or_normal_func, StringType())
dataframe3 = dataFrame.withColumn('label1', udf_attOrNorm(dataFrame.interactions))
dataframe3.show()

IllegalArgumentException: ignored

In [65]:
dataframe3.printSchema()

root
 |-- duration: integer (nullable = true)
 |-- protocol_type: string (nullable = true)
 |-- service: string (nullable = true)
 |-- flag: string (nullable = true)
 |-- src_bytes: integer (nullable = true)
 |-- dst_bytes: integer (nullable = true)
 |-- interactions: string (nullable = true)
 |-- label1: string (nullable = true)



In [66]:
dataframe3.groupby("label1").count().show()

Py4JJavaError: ignored

In [70]:
dataframe3.groupby("label1", "protocol_type").count().show()

Py4JJavaError: ignored